In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import pandas as pd
import pymongo
from splinter import Browser

## Part 1: Web Scraping

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

### NASA Mars News

In [3]:
news_url = "https://mars.nasa.gov/news/"
browser.visit(news_url)

In [4]:
html = browser.html
soup = bs(html, 'html.parser')

In [5]:
news_title = soup.find('div', class_='content_title').a.text
news_p = soup.find('div', class_='article_teaser_body').text

### JPL Mars Space Images

In [6]:
img_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(img_url)

In [7]:
html = browser.html
soup = bs(html, 'html.parser')

In [8]:
base_img_url = "https://www.jpl.nasa.gov"
img_url_snippet = soup.find('a', class_="button fancybox")['data-fancybox-href']
featured_img_url = base_img_url + img_url_snippet

### Mars Weather

In [9]:
weather_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(weather_url)

In [10]:
html = browser.html
soup = bs(html, 'html.parser')

In [11]:
results = soup.find_all("div", class_="js-tweet-text-container")

for result in results:
    tweet = result.p.text
    tweet_split = tweet.split(" ")
    if ('hPa' in tweet_split) or ('temps' in tweet_split):
        break
        
mars_weather = tweet

### Mars Facts

In [12]:
facts_url = "https://space-facts.com/mars/"
browser.visit(facts_url)

In [13]:
tables = pd.read_html(facts_url)
mars_facts = tables[1]
mars_facts.columns = ["description", "value"]
mars_facts = mars_facts.set_index('description')
mars_facts

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


### Mars Hemispheres

In [14]:
hemis_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemis_url)

In [15]:
html = browser.html
soup = bs(html, 'html.parser')

In [16]:
base_url = "https://astrogeology.usgs.gov"
hemis_img_urls = []

results = soup.find_all("div", class_='item')
for result in results:
    
    title = result.find('h3').text
    img_page_url = base_url + result.find('a')['href']
    
    response = requests.get(img_page_url)
    img_soup = bs(response.text, 'html.parser')
    
    img_url = img_soup.find('ul').li.a['href']
    
    img_dict = {'title': title, 
                'img_url': img_url}
    
    hemis_img_urls.append(img_dict)

## Part 2: MongDB and Flask App

### Return Scraped Data as a Dictionary

In [17]:
mars_dict = {'news_title': news_title,
             'news_p': news_p,
             'featured_image_url': featured_img_url,
             'current_weather': mars_weather,
             'mars_facts': mars_facts,
             'hemisphere_image_urls': hemis_img_urls}